* https://python-visualization.github.io/folium/latest/

# 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import folium

# 데이터 불러오기

In [ ]:
df = pd.read_csv('Subway_Stations.csv')

In [ ]:
df.head()

,호선,역명,구주소,도로명주소,위도,경도,Region_Name
0,2호선,봉천,서울특별시 관악구 봉천동 979-7 봉천역(2호선),서울특별시 관악구 남부순환로 지하1721(봉천동),37.487812,126.957234,관악구
1,1호선,서울,서울특별시 중구 남대문로5가 73-6 서울역(1호선),서울특별시 중구 세종대로 지하2(남대문로 5가),37.558735,126.973859,중구
2,1호선,시청,서울특별시 중구 정동 5-5 시청역(1호선),서울특별시 중구 세종대로 지하101(정동),37.566608,126.973148,중구
3,1호선,종각,서울특별시 종로구 종로1가 54 종각역(1호선),서울특별시 종로구 종로 지하55(종로1가),37.570011,126.982108,종로구
4,1호선,종로3가,서울특별시 종로구 종로3가 10-5 종로3가역(1호선),서울특별시 종로구 종로 지하129(종로3가),37.570447,126.990896,종로구


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   호선           273 non-null    object 
 1   역명           273 non-null    object 
 2   구주소          273 non-null    object 
 3   도로명주소        273 non-null    object 
 4   위도           273 non-null    float64
 5   경도           273 non-null    float64
 6   Region_Name  273 non-null    object 
dtypes: float64(2), object(5)
memory usage: 15.1+ KB


# 지도 시각화

## folium 마커(Marker)

In [ ]:
# 지도의 중심을 지정하기 위해 위도와 경도의 평균을 구합니다.
lat = df["위도"].mean()
long = df["경도"].mean()

(lat, long)

(37.54812104139194, 127.00911628534799)

In [ ]:
# icon=folium.Icon(color=icon_color) 로 아이콘 컬러를 변경합니다.

m = folium.Map([lat, long])

for i in df.index:
    sub_lat = df.loc[i, "위도"]
    sub_long = df.loc[i, "경도"]
#     title = df.loc[i, "상호명"] + " - " + df.loc[i, "도로명주소"]

    icon_color = "blue"

    folium.Marker(
        [sub_lat, sub_long],
        icon = folium.Icon(color=icon_color)
        ).add_to(m)

# m.save('index.html')
m

## folium 자치구별

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
Region_Name = df['Region_Name'].value_counts()
Region_Name

송파구     28
중구      23
강남구     21
마포구     16
종로구     15
강동구     14
성동구     14
은평구     13
노원구     13
서초구     12
동작구     12
광진구     11
영등포구    11
용산구     10
강서구      9
성북구      9
중랑구      8
구로구      7
동대문구     6
양천구      5
서대문구     5
관악구      4
강북구      3
도봉구      3
금천구      1
Name: Region_Name, dtype: int64

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=10,
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=Region_Name,
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 지하철역 수"
            )
m

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=10,
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=Region_Name,
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 지하철역 수"
            )

for i in df.index:
    sub_lat = df.loc[i, "위도"]
    sub_long = df.loc[i, "경도"]
#     title = df.loc[i, "상호명"] + " - " + df.loc[i, "도로명주소"]

    icon_color = "gray"
    radius = 200

    folium.CircleMarker(
        [sub_lat,sub_long ],
        radius=1,
        color=icon_color,
#         popup=f'<i>{title}</i>',
#         tooltip=title
    ).add_to(m)

m

## folium 직선

In [ ]:
m = folium.Map(
    location = [37.54812104139194, 127.00911628534799],
    zoom_start = 10
)

# tooltip = 'Click!!'

for i in range(df.shape[0]):
    folium.Marker(
        [df.iloc[i]['위도'], df.iloc[i]['경도']],
    ).add_to(m)


    folium.PolyLine(
        locations = [[37.54812104139194, 127.00911628534799], [df.iloc[i]['위도'], df.iloc[i]['경도']]],
        # tooltip = tooltip
    ).add_to(m)

m

## folium 거리

In [ ]:
!pip install haversine

In [ ]:
from haversine import haversine

In [ ]:
m = folium.Map(
    location = [37.54812104139194, 127.00911628534799],
    zoom_start = 12
)

tooltip = 'Click!!'

for i in range(df.shape[0]):
    folium.Marker(
        [df.iloc[i]['위도'], df.iloc[i]['경도']],
        # popup = f'{df.iloc[i]["오름명"]}\
        # tooltip = tooltip
        tooltip = f'현재위치에서 {df.iloc[i]["역명"]}까지 {거리:.2f}km'
    ).add_to(m)

    # folium.Circle(
    #     [df.iloc[i]['위도'], df.iloc[i]['경도']],
    #     radius = df.iloc[i]['표고(m)'],
    #     color = 'red',
    #     fill = True
    # ).add_to(m)

    location_one = [37.54812104139194, 127.00911628534799]
    location_two = [df.iloc[i]['위도'], df.iloc[i]['경도']]
    거리 = haversine(location_one, location_two, unit = 'km')

    folium.PolyLine(
        locations = [location_one, location_two],
    ).add_to(m)

m

## folium 위도 경도 클릭

In [ ]:
m = folium.Map(location=[37.54812104139194, 127.00911628534799],
               zoom_start=11)

m.add_child(folium.LatLngPopup())

In [ ]:
import folium
from folium.plugins import MousePosition


m = folium.Map()

MousePosition().add_to(m)

m

In [ ]:
m = folium.Map()

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position="topright",
    separator=" | ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m)

m

## folium 원그리기

In [ ]:
m = folium.Map(location=[37.54812104139194, 127.00911628534799],
               zoom_start=11)

# Circle
folium.Circle(
    location=[37.54812104139194, 127.00911628534799],
    radius=100, # 원 크기
    color='#eb9e34', # 원 선 색상
    fill_color='red', # 원 내부 색상
    popup='Circle popup',
    tooltip='Circle tooltip'
).add_to(m)

# CircleMarker
folium.CircleMarker(
    [37.54812104139194, 127.00911628534799],
    radius=100,
    fill=True,
    color='#34ebc6',
    fill_color='#c634eb',
    popup='CircleMarker popup',
    tooltip='CircleMarker tooltip'
).add_to(m)

m

In [ ]:
m = folium.Map(location=[37.54812104139194, 127.00911628534799],
               zoom_start=15)

# Circle
folium.Circle(
    location=[37.54812104139194, 127.00911628534799],
    radius=500, # 원 크기
    color='#eb9e34', # 원 선 색상
    fill_color='red', # 원 내부 색상
    popup='Circle popup',
    tooltip='Circle tooltip'
).add_to(m)

# # CircleMarker
# folium.CircleMarker(
#     [37.54812104139194, 127.00911628534799],
#     radius=100,
#     fill=True,
#     color='#34ebc6',
#     fill_color='#c634eb',
#     # popup='CircleMarker popup',
#     # tooltip='CircleMarker tooltip'
# ).add_to(m)

m
# m.add_child(folium.LatLngPopup())

In [ ]:
m = folium.Map(location=[37.54812104139194, 127.00911628534799],
               zoom_start=11)

m.add_child(folium.LatLngPopup())

## folium save

In [ ]:
# m.save('index.html')
# folium.save('path')